In [1]:
from utils import read_yahoo
starts=['2000-01-01']*2
ends=[None]*2
tickers = ('GOOG', 'AAPL')
df = read_yahoo(tickers, starts, ends)

[Warning] requested start data (2000-01-01) for GOOG != provided start data (2004-08-19T00:00:00.000000000)
[Warning] requested start data (2000-01-01) for AAPL != provided start data (2000-01-03T00:00:00.000000000)


In [2]:
from utils import calc_pct_change_per_col
df = calc_pct_change_per_col(df)
df

,High,Low,Open,Close,Volume,Adj Close,Ticker,Date,High Change,Low Change,Open Change,Close Change,Volume Change,Adj Close Change
1,54.336334,50.062355,50.316402,53.952770,22942874.0,53.952770,GOOG,2004-08-20,0.048241,0.047311,0.010100,0.079430,-0.488697,0.079430
2,56.528118,54.321388,55.168217,54.495735,18342897.0,54.495735,GOOG,2004-08-23,0.040337,0.085075,0.096426,0.010064,-0.200497,0.010064
3,55.591629,51.591621,55.412300,52.239197,15319808.0,52.239197,GOOG,2004-08-24,-0.016567,-0.050252,0.004424,-0.041408,-0.164810,-0.041408
4,53.798351,51.746044,52.284027,52.802086,9232276.0,52.802086,GOOG,2004-08-25,-0.032258,0.002993,-0.056454,0.010775,-0.397363,0.010775
5,53.773445,52.134586,52.279045,53.753517,7128620.0,53.753517,GOOG,2004-08-26,-0.000463,0.007509,-0.000095,0.018019,-0.227859,0.018019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9925,172.500000,168.169998,169.080002,172.190002,106765600.0,172.190002,AAPL,2022-01-10,-0.009418,-0.016722,-0.022037,0.000116,0.233142,0.000116
9926,175.179993,170.820007,172.320007,175.080002,76138300.0,175.080002,AAPL,2022-01-11,0.015536,0.015758,0.019163,0.016784,-0.286865,0.016784
9927,177.179993,174.820007,176.119995,175.529999,74805200.0,175.529999,AAPL,2022-01-12,0.011417,0.023416,0.022052,0.002570,-0.017509,0.002570
9928,176.619995,171.789993,175.779999,172.190002,84505800.0,172.190002,AAPL,2022-01-13,-0.003161,-0.017332,-0.001930,-0.019028,0.129678,-0.019028


In [72]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

nbins = 100
save = True
bin_edges = np.linspace(-0.5, 0.5, 101)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

cols = df.columns.tolist()
skip_cols = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker', 'Date']
for skip_col in skip_cols:
    cols.remove(skip_col)

tickers = df['Ticker'].unique().tolist()
ticker_df_dict = {ticker : pd.DataFrame for ticker in tickers}
for k in ticker_df_dict.keys():
    ticker_df_dict[k] = df[df['Ticker'] == k]

for col in cols:
    for k, v in ticker_df_dict.items():
        mean = v[col].mean()
        std = v[col].std()
        pos_prob = (v[col].to_numpy() > 0).sum() / v[col].to_numpy().shape[0]
        neg_prob = (v[col].to_numpy() < 0).sum() / v[col].to_numpy().shape[0]
        mean_pos = v[col].to_numpy()[v[col].to_numpy() > 0].mean()
        mean_neg = v[col].to_numpy()[v[col].to_numpy() < 0].mean()
        title = '{}, mean={:.4f}, std={:.4f}'.format(k, mean, std)  
        # fig = px.histogram(v, x=col, nbins=nbins, title=title, histnorm='probability')#'density')#'probability density')
        counts, _ = np.histogram(v[col], bins=bin_edges)
        probs = counts / counts.sum()
        fig = px.bar(x=bin_centers, y=probs, labels={'x': col, 'y':'probability'}, title=title)
        if save:
            name = os.path.join('pct_change', k, col.split()[0] + '.html')
            os.makedirs(os.path.dirname(name), exist_ok=True)
            fig.write_html(name)
        fig.show()
        print('{}\t{}\t|\tmean={:.4f}\tstd={:.4f}\tpos_prob={:.4f}\tneg_prob={:.4f}\tmean_pos={:.4f}\tmean_neg={:.4f}'.format(k, col, mean, std, pos_prob, neg_prob, mean_pos, mean_neg))
        

GOOG	High Change	|	mean=0.0010	std=0.0167	pos_prob=0.5159	neg_prob=0.4835	mean_pos=0.0115	mean_neg=-0.0101


AAPL	High Change	|	mean=0.0012	std=0.0217	pos_prob=0.5178	neg_prob=0.4772	mean_pos=0.0147	mean_neg=-0.0135


GOOG	Low Change	|	mean=0.0011	std=0.0178	pos_prob=0.5537	neg_prob=0.4454	mean_pos=0.0117	mean_neg=-0.0121


AAPL	Low Change	|	mean=0.0012	std=0.0238	pos_prob=0.5592	neg_prob=0.4361	mean_pos=0.0150	mean_neg=-0.0164


GOOG	Open Change	|	mean=0.0011	std=0.0192	pos_prob=0.5284	neg_prob=0.4702	mean_pos=0.0133	mean_neg=-0.0126


...

AAPL	Close Change	|	mean=0.0013	std=0.0252	pos_prob=0.5214	neg_prob=0.4734	mean_pos=0.0175	mean_neg=-0.0166


GOOG	Volume Change	|	mean=0.0762	std=0.7048	pos_prob=0.4778	neg_prob=0.5222	mean_pos=0.3920	mean_neg=-0.2127


AAPL	Volume Change	|	mean=0.0708	std=0.4771	pos_prob=0.4745	neg_prob=0.5255	mean_pos=0.3828	mean_neg=-0.2108


GOOG	Adj Close Change	|	mean=0.0011	std=0.0190	pos_prob=0.5266	neg_prob=0.4732	mean_pos=0.0131	mean_neg=-0.0122


AAPL	Adj Close Change	|	mean=0.0013	std=0.0252	pos_prob=0.5221	neg_prob=0.4727	mean_pos=0.0175	mean_neg=-0.0165
